### Import necessary spark and python stuff

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data
*path_on_hdfs_to_your_data*

In [3]:
df = spark.read.json("/user/rucio01/traces/traces.2018-04*")

### check the structure

In [4]:
# df.printSchema()

### Filter for loadAvg
_https://metricmgr.cern.ch/metric/20002/_

In [5]:
df.head()

Row(account=u'pilot', appid=None, catStart=None, clientState=u'DONE', dataset=u'', datasetName=None, datasetScope=u'', dataset_scope=None, duid=None, eventType=u'download', eventVersion=u'1.14.11', filename=u'EventService_premerge_3886111819_13607623-3886111819-13041798597-3795-10.tar', filesize=None, guid=None, hostname=u'b6ac037cc1.cern.ch', ip=None, localSite=None, name=None, pq=None, protocol=u's3', relativeStart=None, remoteSite=u'CERN-PROD_ES', scope=u'transient', stateReason=None, suspicious=None, taskid=None, timeEnd=None, timeStart=None, traceId=u'e68c1b1bbee84d5a96dcfc2f38dc26bb', traceIp=u'::ffff:188.184.146.18', traceTimeentry=u'2018-03-31T22:00:00.005599', traceTimeentryUnix=1522533600.005599, transferEnd=1522533599.975359, transferStart=1522533588.042424, url=None, usr=None, usrdn=None, uuid=u'53cf181ec38b4e2a8885b2a81109d505', validateStart=None, version=None)

### Create temporary table view

In [6]:
# body_schema = spark.read.json(df_loadAvg.rdd.map(lambda row: row.body)).schema
df.createOrReplaceTempView("traces")

### Use PySpark to query download events and split dataset DID into new features

In [7]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [8]:
df_ps = spark.sql("SELECT account, clientState, dataset, eventType, eventVersion, filename, filesize, remoteSite, traceTimeentryUnix\
                               FROM traces \
                               WHERE dataset != 'None' AND dataset != '' AND dataset != '[null]' AND (dataset LIKE 'data%' OR dataset LIKE 'mc%')")\

In [9]:
df_ps.show(n=5)

+-------+-----------+--------------------+-----------+------------+--------------------+----------+--------------------+-------------------+
|account|clientState|             dataset|  eventType|eventVersion|            filename|  filesize|          remoteSite| traceTimeentryUnix|
+-------+-----------+--------------------+-----------+------------+--------------------+----------+--------------------+-------------------+
|   null|       DONE|mc15_13TeV.364250...|put_sm_logs|      pilot3|log.13551031._142...|  82650890|       NET2_DATADISK|1.522533600010634E9|
|   null|       DONE|mc16_13TeV:mc16_1...|     get_sm|      pilot3|EVNT.12860069._00...| 447515727|   RAL-LCG2_DATADISK|1.522533600038918E9|
|   null|       DONE|data16_13TeV:data...|     get_sm|      pilot3|AOD.11192600._007...|1627225437|SARA-MATRIX_DATADISK| 1.52253360005325E9|
|   null|       DONE|data15_13TeV:data...|   get_sm_a|      pilot3|AOD.07709524._000...|4211382595|   IN2P3-CC_DATADISK|1.522533600083759E9|
|   null|    

In [10]:
# df_nulls = df_ps.where(col("account").isNull())

In [11]:
# df_nulls.show(n=5)

In [12]:
# replacing null values in column account with the value of column eventVersion of the same row
# df_ps2 = df_ps.withColumn("account",coalesce(df.account,df.eventVersion))
# df_ps2.show(n=5)

df_ps = df_ps.dropna()
df_ps.show(n=5)

+--------+------------------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+
| account|       clientState|             dataset|eventType|eventVersion|            filename|  filesize|          remoteSite| traceTimeentryUnix|
+--------+------------------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+
|    yliu|ServiceUnavailable|data15_13TeV.0028...| download|     1.14.11|DAOD_HIGG5D1.1156...| 416900305|TAIWAN-LCG2_DATADISK|1.522533840149492E9|
|    yliu|              DONE|data15_13TeV.0028...| download|     1.14.11|DAOD_HIGG5D1.1156...| 358630843|       NET2_DATADISK|1.522533840253131E9|
|    yliu|ServiceUnavailable|data15_13TeV.0028...| download|     1.14.11|DAOD_HIGG5D1.1156...| 417994016|TAIWAN-LCG2_DATADISK|1.522533914030552E9|
|changqia|              DONE|data16_13TeV.0030...| download|     1.14.11|DAOD_HIGG5D1.1261...|3998468117|   IN2P3-CC_D

In [13]:
split_col = split(df_ps['dataset'], "\.")
df_ps = df_ps.withColumn('project', split_col.getItem(0))
df_ps = df_ps.withColumn('run_number', split_col.getItem(1))
df_ps = df_ps.withColumn('stream_name', split_col.getItem(2))
df_ps = df_ps.withColumn('prod_step', split_col.getItem(3))
df_ps = df_ps.withColumn('datatype', split_col.getItem(4))
df_ps = df_ps.withColumn('dataset_version', split_col.getItem(5))
df_ps.show(n=5)


+--------+------------------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+------------+----------+------------+---------+------------+-----------------+
| account|       clientState|             dataset|eventType|eventVersion|            filename|  filesize|          remoteSite| traceTimeentryUnix|     project|run_number| stream_name|prod_step|    datatype|  dataset_version|
+--------+------------------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+------------+----------+------------+---------+------------+-----------------+
|    yliu|ServiceUnavailable|data15_13TeV.0028...| download|     1.14.11|DAOD_HIGG5D1.1156...| 416900305|TAIWAN-LCG2_DATADISK|1.522533840149492E9|data15_13TeV|  00280231|physics_Main|    deriv|DAOD_HIGG5D1|r9264_p3083_p3213|
|    yliu|              DONE|data15_13TeV.0028...| download|     1.14.11|DAOD_HIGG5D1.1156...| 35863

### Perform One-Hot-encoding to make categorical data numerical (necessary for linear regression)

In [14]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
# https://spark.apache.org/docs/2.1.0/ml-features.html#onehotencoder


In [15]:

categoricalFeatures = ['account', 'clientState', 'dataset', 'eventType', 'eventVersion', 'filename', 'remoteSite', 'project', 'stream_name', 'prod_step', 'datatype'] # feature dataset version causes error: "Caused by: org.apache.spark.SparkException: StringIndexer encountered NULL value. To handle or skip NULLS, try setting StringIndexer.handleInvalid."

encoded = df_ps

for elem in categoricalFeatures:
    stringIndexer = StringIndexer(inputCol=elem, outputCol= elem + "Index")
    model = stringIndexer.fit(encoded)
    indexed = model.transform(encoded)
    
    if indexed.select(elem + 'Index').distinct().count() > 1:
        encoder = OneHotEncoder(inputCol= elem + "Index", outputCol= elem + "Vec")
        encoded = encoder.transform(indexed)

encoded.show(n=5)

+--------+------------------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+------------+----------+------------+---------+------------+-----------------+------------+----------------+----------------+--------------+------------+--------------------+-----------------+---------------+-------------+--------------------+---------------+----------------+------------+--------------+----------------+----------------+--------------+-------------+-------------+----------------+
| account|       clientState|             dataset|eventType|eventVersion|            filename|  filesize|          remoteSite| traceTimeentryUnix|     project|run_number| stream_name|prod_step|    datatype|  dataset_version|accountIndex|      accountVec|clientStateIndex|clientStateVec|datasetIndex|          datasetVec|eventVersionIndex|eventVersionVec|filenameIndex|         filenameVec|remoteSiteIndex|   remoteSiteVec|projectIndex|    projectVec|strea

In [16]:
encoded.createOrReplaceTempView("encoded")

In [17]:
# TODO: linear regression like here: https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning#basics

df_ps_counts = spark.sql("SELECT dataset AS dataset_counts, count(*) as hits \
                               FROM traces \
                               WHERE dataset != 'None' AND dataset != '' AND dataset != '[null]' AND (dataset LIKE 'data%' OR dataset LIKE 'mc%')\
                               GROUP BY  dataset \
                               ORDER BY 2 DESC")
df_ps_counts.show(n=5)

+--------------------+-------+
|      dataset_counts|   hits|
+--------------------+-------+
|mc16_13TeV:mc16_1...|1640973|
|mc16_13TeV:mc16_1...| 985279|
|mc15_13TeV:mc15_1...| 796275|
|mc15_13TeV:mc15_1...| 791602|
|mc15_13TeV:mc15_1...| 465155|
+--------------------+-------+
only showing top 5 rows



In [18]:
df_ps_counts.createOrReplaceTempView("counts")

In [19]:
df_ps_merged = spark.sql("SELECT * \
                               FROM encoded \
                               INNER JOIN counts ON encoded.dataset = counts.dataset_counts")

In [20]:
df_ps_merged.show(n=5)

+-------+-----------+--------------------+---------+------------+--------------------+---------+--------------------+-------------------+------------+----------+------------+---------+------------+-----------------+------------+---------------+----------------+--------------+------------+-------------------+-----------------+---------------+-------------+--------------------+---------------+----------------+------------+--------------+----------------+----------------+--------------+-------------+-------------+---------------+--------------------+----+
|account|clientState|             dataset|eventType|eventVersion|            filename| filesize|          remoteSite| traceTimeentryUnix|     project|run_number| stream_name|prod_step|    datatype|  dataset_version|accountIndex|     accountVec|clientStateIndex|clientStateVec|datasetIndex|         datasetVec|eventVersionIndex|eventVersionVec|filenameIndex|         filenameVec|remoteSiteIndex|   remoteSiteVec|projectIndex|    projectVec|st

In [21]:
df_ps_merged.count()

781748

In [22]:
df_ps_merged.createOrReplaceTempView("mergedCheck")
test = spark.sql("SELECT dataset, dataset_counts \
                               FROM mergedCheck \
                               WHERE dataset = dataset_counts")

In [23]:
test.show(n=5)

+--------------------+--------------------+
|             dataset|      dataset_counts|
+--------------------+--------------------+
|data15_13TeV.0028...|data15_13TeV.0028...|
|data15_13TeV.0028...|data15_13TeV.0028...|
|data15_13TeV.0028...|data15_13TeV.0028...|
|data15_13TeV.0028...|data15_13TeV.0028...|
|data15_13TeV.0028...|data15_13TeV.0028...|
+--------------------+--------------------+
only showing top 5 rows



In [24]:
test.count()

781748

In [25]:
df_ps_merged = df_ps_merged.drop('dataset_counts')

In [26]:
# refresh merged view so that all columns are in it
df_ps_merged.createOrReplaceTempView("merged")

In [27]:
df_ps_counts.count()

885651

In [28]:
encoded.count()

781748

In [29]:
df_ps_merged.count()

781748

In [30]:
temp_class = spark.sql("SELECT dataset_counts, CASE WHEN hits > 10 THEN 1 ELSE 0 END AS cache FROM counts")\

In [31]:
temp_class.show(n=5)

+--------------------+-----+
|      dataset_counts|cache|
+--------------------+-----+
|mc16_13TeV:mc16_1...|    1|
|mc16_13TeV:mc16_1...|    1|
|mc15_13TeV:mc15_1...|    1|
|mc15_13TeV:mc15_1...|    1|
|mc15_13TeV:mc15_1...|    1|
+--------------------+-----+
only showing top 5 rows



In [32]:
temp_class.createOrReplaceTempView("temp_classes")

In [33]:
df_ps_merged = spark.sql("SELECT * \
                               FROM merged \
                               JOIN temp_classes ON merged.dataset = temp_classes.dataset_counts")

In [34]:
df_ps_merged.show(n=5)

+-------+-----------+--------------------+---------+------------+--------------------+---------+--------------------+-------------------+------------+----------+------------+---------+------------+-----------------+------------+---------------+----------------+--------------+------------+-------------------+-----------------+---------------+-------------+--------------------+---------------+----------------+------------+--------------+----------------+----------------+--------------+-------------+-------------+---------------+----+--------------------+-----+
|account|clientState|             dataset|eventType|eventVersion|            filename| filesize|          remoteSite| traceTimeentryUnix|     project|run_number| stream_name|prod_step|    datatype|  dataset_version|accountIndex|     accountVec|clientStateIndex|clientStateVec|datasetIndex|         datasetVec|eventVersionIndex|eventVersionVec|filenameIndex|         filenameVec|remoteSiteIndex|   remoteSiteVec|projectIndex|    project

In [35]:
df_ps_merged.count()

781748

In [36]:
df_ps_merged_dropped = df_ps_merged.drop('account','clientState','dataset','eventType','eventVersion','filename','remoteSite','project','stream_name','prod_step','datatype','dataset_version', 'dataset_counts','accountIndex','clientStateIndex','datasetIndex','eventTypeIndex','eventVersionIndex','filenameIndex','remoteSiteIndex','projectIndex','stream_nameIndex','prod_stepIndex','datatypeIndex')
df_ps_merged_dropped.show(n=5)

+---------+-------------------+----------+---------------+--------------+-------------------+---------------+--------------------+----------------+--------------+----------------+-------------+---------------+----+-----+
| filesize| traceTimeentryUnix|run_number|     accountVec|clientStateVec|         datasetVec|eventVersionVec|         filenameVec|   remoteSiteVec|    projectVec|  stream_nameVec| prod_stepVec|    datatypeVec|hits|cache|
+---------+-------------------+----------+---------------+--------------+-------------------+---------------+--------------------+----------------+--------------+----------------+-------------+---------------+----+-----+
|191405239|1.523619796151299E9|  00280862|(541,[1],[1.0])|(10,[0],[1.0])|(10591,[827],[1.0])|  (5,[0],[1.0])|(532006,[295245],...| (165,[1],[1.0])|(22,[4],[1.0])|(2956,[0],[1.0])|(8,[0],[1.0])|(111,[3],[1.0])| 250|    1|
|191435598|1.523619811356084E9|  00280862|(541,[1],[1.0])|(10,[0],[1.0])|(10591,[827],[1.0])|  (5,[0],[1.0])|(532006

In [37]:
df_ps_merged_dropped.printSchema()

root
 |-- filesize: string (nullable = true)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- run_number: string (nullable = true)
 |-- accountVec: vector (nullable = true)
 |-- clientStateVec: vector (nullable = true)
 |-- datasetVec: vector (nullable = true)
 |-- eventVersionVec: vector (nullable = true)
 |-- filenameVec: vector (nullable = true)
 |-- remoteSiteVec: vector (nullable = true)
 |-- projectVec: vector (nullable = true)
 |-- stream_nameVec: vector (nullable = true)
 |-- prod_stepVec: vector (nullable = true)
 |-- datatypeVec: vector (nullable = true)
 |-- hits: long (nullable = false)
 |-- cache: integer (nullable = false)



In [38]:
df_ps_final = df_ps_merged_dropped.withColumn("filesize_double",df_ps_merged_dropped.filesize.cast(DoubleType()))
df_ps_final = df_ps_final.withColumn("run_number_double",df_ps_merged_dropped.run_number.cast(DoubleType()))



In [39]:
df_ps_final.printSchema()

root
 |-- filesize: string (nullable = true)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- run_number: string (nullable = true)
 |-- accountVec: vector (nullable = true)
 |-- clientStateVec: vector (nullable = true)
 |-- datasetVec: vector (nullable = true)
 |-- eventVersionVec: vector (nullable = true)
 |-- filenameVec: vector (nullable = true)
 |-- remoteSiteVec: vector (nullable = true)
 |-- projectVec: vector (nullable = true)
 |-- stream_nameVec: vector (nullable = true)
 |-- prod_stepVec: vector (nullable = true)
 |-- datatypeVec: vector (nullable = true)
 |-- hits: long (nullable = false)
 |-- cache: integer (nullable = false)
 |-- filesize_double: double (nullable = true)
 |-- run_number_double: double (nullable = true)



In [40]:
df_ps_final.show(n=5)

+---------+-------------------+----------+---------------+--------------+-------------------+---------------+--------------------+----------------+--------------+----------------+-------------+---------------+----+-----+---------------+-----------------+
| filesize| traceTimeentryUnix|run_number|     accountVec|clientStateVec|         datasetVec|eventVersionVec|         filenameVec|   remoteSiteVec|    projectVec|  stream_nameVec| prod_stepVec|    datatypeVec|hits|cache|filesize_double|run_number_double|
+---------+-------------------+----------+---------------+--------------+-------------------+---------------+--------------------+----------------+--------------+----------------+-------------+---------------+----+-----+---------------+-----------------+
|191405239|1.523619796151299E9|  00280862|(541,[1],[1.0])|(10,[0],[1.0])|(10591,[827],[1.0])|  (5,[0],[1.0])|(532006,[295245],...| (165,[1],[1.0])|(22,[4],[1.0])|(2956,[0],[1.0])|(8,[0],[1.0])|(111,[3],[1.0])| 250|    1|   1.91405239E8

In [41]:
df_ps_final = df_ps_final.drop('run_number', 'filesize')

In [42]:
df_ps_final = df_ps_final.select("cache","traceTimeentryUnix","accountVec","clientStateVec","datasetVec","eventVersionVec","filenameVec","remoteSiteVec","projectVec","stream_nameVec","prod_stepVec","datatypeVec","hits","filesize_double","run_number_double")


In [43]:
df_ps_final.show(n=5)

+-----+-------------------+---------------+--------------+-------------------+---------------+--------------------+----------------+--------------+----------------+-------------+---------------+----+---------------+-----------------+
|cache| traceTimeentryUnix|     accountVec|clientStateVec|         datasetVec|eventVersionVec|         filenameVec|   remoteSiteVec|    projectVec|  stream_nameVec| prod_stepVec|    datatypeVec|hits|filesize_double|run_number_double|
+-----+-------------------+---------------+--------------+-------------------+---------------+--------------------+----------------+--------------+----------------+-------------+---------------+----+---------------+-----------------+
|    1|1.523619796151299E9|(541,[1],[1.0])|(10,[0],[1.0])|(10591,[827],[1.0])|  (5,[0],[1.0])|(532006,[295245],...| (165,[1],[1.0])|(22,[4],[1.0])|(2956,[0],[1.0])|(8,[0],[1.0])|(111,[3],[1.0])| 250|   1.91405239E8|         280862.0|
|    1|1.523619811356084E9|(541,[1],[1.0])|(10,[0],[1.0])|(10591

In [44]:
df_ps_final.printSchema()

root
 |-- cache: integer (nullable = false)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- accountVec: vector (nullable = true)
 |-- clientStateVec: vector (nullable = true)
 |-- datasetVec: vector (nullable = true)
 |-- eventVersionVec: vector (nullable = true)
 |-- filenameVec: vector (nullable = true)
 |-- remoteSiteVec: vector (nullable = true)
 |-- projectVec: vector (nullable = true)
 |-- stream_nameVec: vector (nullable = true)
 |-- prod_stepVec: vector (nullable = true)
 |-- datatypeVec: vector (nullable = true)
 |-- hits: long (nullable = false)
 |-- filesize_double: double (nullable = true)
 |-- run_number_double: double (nullable = true)



### Separating the features from the target variable

In [45]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["traceTimeentryUnix","accountVec","clientStateVec","datasetVec","eventVersionVec","filenameVec","remoteSiteVec","projectVec","stream_nameVec","prod_stepVec","datatypeVec","hits","filesize_double","run_number_double"],
    outputCol="features")

#linearRegression will look for a column called features

output = assembler.transform(df_ps_final)

In [46]:
output.show(n=5)

+-----+-------------------+---------------+--------------+-------------------+---------------+--------------------+----------------+--------------+----------------+-------------+---------------+----+---------------+-----------------+--------------------+
|cache| traceTimeentryUnix|     accountVec|clientStateVec|         datasetVec|eventVersionVec|         filenameVec|   remoteSiteVec|    projectVec|  stream_nameVec| prod_stepVec|    datatypeVec|hits|filesize_double|run_number_double|            features|
+-----+-------------------+---------------+--------------+-------------------+---------------+--------------------+----------------+--------------+----------------+-------------+---------------+----+---------------+-----------------+--------------------+
|    1|1.523620486545565E9|(541,[1],[1.0])|(10,[0],[1.0])|(10591,[827],[1.0])|  (5,[0],[1.0])|(532006,[454482],...|(165,[26],[1.0])|(22,[4],[1.0])|(2956,[0],[1.0])|(8,[0],[1.0])|(111,[3],[1.0])| 250|   1.71292777E8|         280862.0|(5

### Split the data into train and test sets

In [53]:
output_sample = output.sample(False,0.5,1234).limit(10000).select("features","cache")

output_sample.count()

10000

In [57]:
output_sample.show(n=5)

Py4JJavaError: An error occurred while calling o608.showString.
: java.lang.OutOfMemoryError: Java heap space


In [55]:
train_data, test_data = output_sample.randomSplit([.8,.2],seed=1234)

Py4JJavaError: An error occurred while calling o619.count.
: java.lang.OutOfMemoryError: Java heap space


In [50]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="cache", maxIter=10, regParam=0.3, elasticNetParam=0.8)



Py4JJavaError: An error occurred while calling o511.fit.
: java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1786)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1189)
	at java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:348)
	at org.apache.spark.serializer.JavaSerializationStream.writeObject(JavaSerializer.scala:43)
	at org.apache.spark.serializer.JavaSerializerInstance.serialize(JavaSerializer.scala:100)
	at org.apache.spark.

In [ ]:
linearModel = lr.fit(train_data)